##### Copyright 2019 The TensorFlow Authors.

In [1]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Keras overview

<table class="tfo-notebook-buttons" align="left">
  <td>
    <a target="_blank" href="https://www.tensorflow.org/guide/keras/overview"><img src="https://www.tensorflow.org/images/tf_logo_32px.png" />View on TensorFlow.org</a>
  </td>
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs/blob/master/site/en/guide/keras/overview.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />Run in Google Colab</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/tensorflow/docs/blob/master/site/en/guide/keras/overview.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />View source on GitHub</a>
  </td>
  <td>
    <a href="https://storage.googleapis.com/tensorflow_docs/docs/site/en/guide/keras/overview.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png" />Download notebook</a>
  </td>
</table>

This guide gives you the basics to get started with Keras. It's a 10-minute read.

## Import tf.keras

`tf.keras` is TensorFlow's implementation of the
[Keras API specification](https://keras.io). This is a high-level
API to build and train models that includes first-class support for
TensorFlow-specific functionality, such as [eager execution](../eager.ipynb),
`tf.data` pipelines, and [Estimators](../estimator.ipynb).
`tf.keras` makes TensorFlow easier to use without sacrificing flexibility and
performance.

To get started, import `tf.keras` as part of your TensorFlow program setup:

In [2]:
import tensorflow as tf

from tensorflow import keras

`tf.keras` can run any Keras-compatible code, but keep in mind:

* The `tf.keras` version in the latest TensorFlow release might not be the same
  as the latest `keras` version from PyPI. Check `tf.keras.__version__`.
* When [saving a model's weights](./save_and_serialize.ipynb), `tf.keras` defaults to the
  [checkpoint format](../checkpoint.ipynb). Pass `save_format='h5'` to
  use HDF5 (or pass a filename that ends in `.h5`).

## Build a simple model

### Sequential model

In Keras, you assemble *layers* to build *models*. A model is (usually) a graph
of layers. The most common type of model is a stack of layers: the
`tf.keras.Sequential` model.

To build a simple, fully-connected network (i.e. multi-layer perceptron):

In [3]:
from tensorflow.keras import layers

model = tf.keras.Sequential()
# Adds a densely-connected layer with 64 units to the model:
model.add(layers.Dense(64, activation='relu'))
# Add another:
model.add(layers.Dense(64, activation='relu'))
# Add an output layer with 10 output units:
model.add(layers.Dense(10))

You can find a complete, short example of how to use Sequential models [here](https://www.tensorflow.org/tutorials/quickstart/beginner).

To learn about building more advanced models than Sequential models, see:
- [Guide to the Keras Functional API](./functional.ipynb)
- [Guide to writing layers and models from scratch with subclassing](./custom_layers_and_models.ipynb)

### Configure the layers

There are many `tf.keras.layers` available. Most of them share some common constructor
arguments:

* `activation`: Set the activation function for the layer. This parameter is
  specified by the name of a built-in function or as a callable object. By
  default, no activation is applied.
* `kernel_initializer` and `bias_initializer`: The initialization schemes
  that create the layer's weights (kernel and bias). This parameter is a name or
  a callable object. The kernel defaults to the `"Glorot uniform"` initializer, 
  and the bias defaults to zeros.
* `kernel_regularizer` and `bias_regularizer`: The regularization schemes
  that apply the layer's weights (kernel and bias), such as L1 or L2
  regularization. By default, no regularization is applied.

The following instantiates `tf.keras.layers.Dense` layers using constructor
arguments:

In [4]:
# Create a relu layer:
layers.Dense(64, activation='relu')
# Or:
layers.Dense(64, activation=tf.nn.relu)

# A linear layer with L1 regularization of factor 0.01 applied to the kernel matrix:
layers.Dense(64, kernel_regularizer=tf.keras.regularizers.l1(0.01))

# A linear layer with L2 regularization of factor 0.01 applied to the bias vector:
layers.Dense(64, bias_regularizer=tf.keras.regularizers.l2(0.01))

# A linear layer with a kernel initialized to a random orthogonal matrix:
layers.Dense(64, kernel_initializer='orthogonal')

# A linear layer with a bias vector initialized to 2.0s:
layers.Dense(64, bias_initializer=tf.keras.initializers.Constant(2.0))

## Train and evaluate

### Set up training

After the model is constructed, configure its learning process by calling the
`compile` method:

In [5]:
model = tf.keras.Sequential([
# Adds a densely-connected layer with 64 units to the model:
layers.Dense(64, activation='relu', input_shape=(32,)),
# Add another:
layers.Dense(64, activation='relu'),
# Add an output layer with 10 output units:
layers.Dense(10)])

model.compile(optimizer=tf.keras.optimizers.Adam(0.01),
              loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

Instructions for updating:
If using Keras pass *_constraint arguments to layers.


`tf.keras.Model.compile` takes three important arguments:

* `optimizer`: This object specifies the training procedure. Pass it optimizer
  instances from the `tf.keras.optimizers` module, such as
  `tf.keras.optimizers.Adam` or
  `tf.keras.optimizers.SGD`. If you just want to use the default parameters, you can also specify optimizers via strings, such as `'adam'` or `'sgd'`.
* `loss`: The function to minimize during optimization. Common choices include
  mean square error (`mse`), `categorical_crossentropy`, and
  `binary_crossentropy`. Loss functions are specified by name or by
  passing a callable object from the `tf.keras.losses` module.
* `metrics`: Used to monitor training. These are string names or callables from
  the `tf.keras.metrics` module.
* Additionally, to make sure the model trains and evaluates eagerly, you can make sure to pass `run_eagerly=True` as a parameter to compile.


The following shows a few examples of configuring a model for training:

In [6]:
# Configure a model for mean-squared error regression.
model.compile(optimizer=tf.keras.optimizers.Adam(0.01),
              loss='mse',       # mean squared error
              metrics=['mae'])  # mean absolute error

# Configure a model for categorical classification.
model.compile(optimizer=tf.keras.optimizers.RMSprop(0.01),
              loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

### Train from NumPy data

For small datasets, use in-memory [NumPy](https://www.numpy.org/)
arrays to train and evaluate a model. The model is "fit" to the training data
using the `fit` method:

In [6]:
import numpy as np

data = np.random.random((1000, 32))
labels = np.random.random((1000, 10))

model.fit(data, labels, epochs=10, batch_size=32)

Train on 1000 samples
Epoch 1/10
1000/1000 [==============================] - 0s 275us/sample - loss: 57.8783 - acc: 0.0880
Epoch 2/10
1000/1000 [==============================] - 0s 98us/sample - loss: 268.1582 - acc: 0.1060
Epoch 3/10
1000/1000 [==============================] - 0s 113us/sample - loss: 880.1472 - acc: 0.0890
Epoch 4/10
1000/1000 [==============================] - 0s 110us/sample - loss: 2363.2006 - acc: 0.1100
Epoch 5/10
1000/1000 [==============================] - 0s 132us/sample - loss: 5946.8446 - acc: 0.0910
Epoch 6/10
1000/1000 [==============================] - 0s 123us/sample - loss: 10485.1677 - acc: 0.1200
Epoch 7/10
1000/1000 [==============================] - 0s 116us/sample - loss: 12039.1040 - acc: 0.1070
Epoch 8/10
1000/1000 [==============================] - 0s 121us/sample - loss: 19355.8533 - acc: 0.1070
Epoch 9/10
1000/1000 [==============================] - 0s 119us/sample - loss: 24756.1298 - acc: 0.1070
Epoch 10/10
1000/1000 [====================

`tf.keras.Model.fit` takes three important arguments:

* `epochs`: Training is structured into *epochs*. An epoch is one iteration over
  the entire input data (this is done in smaller batches).
* `batch_size`: When passed NumPy data, the model slices the data into smaller
  batches and iterates over these batches during training. This integer
  specifies the size of each batch. Be aware that the last batch may be smaller
  if the total number of samples is not divisible by the batch size.
* `validation_data`: When prototyping a model, you want to easily monitor its
  performance on some validation data. Passing this argument—a tuple of inputs
  and labels—allows the model to display the loss and metrics in inference mode
  for the passed data, at the end of each epoch.

Here's an example using `validation_data`:

In [7]:
import numpy as np

data = np.random.random((1000, 32))
labels = np.random.random((1000, 10))

val_data = np.random.random((100, 32))
val_labels = np.random.random((100, 10))

model.fit(data, labels, epochs=10, batch_size=32,
          validation_data=(val_data, val_labels))

Train on 1000 samples, validate on 100 samples
Epoch 1/10
1000/1000 [==============================] - 0s 186us/sample - loss: 33435.2523 - acc: 0.1050 - val_loss: 26396.3943 - val_acc: 0.0600
Epoch 2/10
1000/1000 [==============================] - 0s 105us/sample - loss: 43837.4126 - acc: 0.1080 - val_loss: 45688.4697 - val_acc: 0.0600
Epoch 3/10
1000/1000 [==============================] - 0s 98us/sample - loss: 59132.8763 - acc: 0.0960 - val_loss: 57897.8402 - val_acc: 0.1100
Epoch 4/10
1000/1000 [==============================] - 0s 109us/sample - loss: 68182.0259 - acc: 0.1160 - val_loss: 94920.4522 - val_acc: 0.1000
Epoch 5/10
1000/1000 [==============================] - 0s 100us/sample - loss: 92510.5814 - acc: 0.0880 - val_loss: 109705.1047 - val_acc: 0.1100
Epoch 6/10
1000/1000 [==============================] - 0s 103us/sample - loss: 90546.2409 - acc: 0.0970 - val_loss: 97987.9044 - val_acc: 0.1100
Epoch 7/10
1000/1000 [==============================] - 0s 116us/sample - los

### Train from tf.data datasets

Use the [Datasets API](../data.ipynb) to scale to large datasets
or multi-device training. Pass a `tf.data.Dataset` instance to the `fit`
method:

In [8]:
# Instantiates a toy dataset instance:
dataset = tf.data.Dataset.from_tensor_slices((data, labels))
dataset = dataset.batch(32)

model.fit(dataset, epochs=10)

Train on 32 steps
Epoch 1/10
32/32 [==============================] - 0s 8ms/step - loss: 113054.1655 - acc: 0.0980
Epoch 2/10
32/32 [==============================] - 0s 4ms/step - loss: 198394.3301 - acc: 0.0940
Epoch 3/10
32/32 [==============================] - 0s 3ms/step - loss: 233805.5723 - acc: 0.0780
Epoch 4/10
32/32 [==============================] - 0s 3ms/step - loss: 221544.5400 - acc: 0.1000
Epoch 5/10
32/32 [==============================] - 0s 3ms/step - loss: 200797.5674 - acc: 0.1060
Epoch 6/10
32/32 [==============================] - 0s 3ms/step - loss: 220190.4014 - acc: 0.0880
Epoch 7/10
32/32 [==============================] - 0s 3ms/step - loss: 277466.6729 - acc: 0.1040
Epoch 8/10
32/32 [==============================] - 0s 3ms/step - loss: 325647.0796 - acc: 0.0960
Epoch 9/10
32/32 [==============================] - 0s 3ms/step - loss: 285330.5845 - acc: 0.1020
Epoch 10/10
32/32 [==============================] - 0s 3ms/step - loss: 307187.1479 - acc: 0.0820


Since the `Dataset` yields batches of data, this snippet does not require a `batch_size`.

Datasets can also be used for validation:

In [9]:
dataset = tf.data.Dataset.from_tensor_slices((data, labels))
dataset = dataset.batch(32)

val_dataset = tf.data.Dataset.from_tensor_slices((val_data, val_labels))
val_dataset = val_dataset.batch(32)

model.fit(dataset, epochs=10,
          validation_data=val_dataset)

Train on 32 steps, validate on 4 steps
Epoch 1/10
32/32 [==============================] - 0s 11ms/step - loss: 324586.3896 - acc: 0.1020 - val_loss: 191016.7656 - val_acc: 0.0600
Epoch 2/10
32/32 [==============================] - 0s 3ms/step - loss: 245081.7437 - acc: 0.1010 - val_loss: 328870.4609 - val_acc: 0.1300
Epoch 3/10
32/32 [==============================] - 0s 3ms/step - loss: 374589.0449 - acc: 0.1080 - val_loss: 275045.9922 - val_acc: 0.1100
Epoch 4/10
32/32 [==============================] - 0s 3ms/step - loss: 387798.9004 - acc: 0.1110 - val_loss: 192681.8320 - val_acc: 0.1100
Epoch 5/10
32/32 [==============================] - 0s 4ms/step - loss: 326535.4663 - acc: 0.0910 - val_loss: 390813.0391 - val_acc: 0.0600
Epoch 6/10
32/32 [==============================] - 0s 4ms/step - loss: 391437.2100 - acc: 0.1110 - val_loss: 582690.6875 - val_acc: 0.0900
Epoch 7/10
32/32 [==============================] - 0s 3ms/step - loss: 453599.6357 - acc: 0.0900 - val_loss: 299087.765

### Evaluate and predict

The `tf.keras.Model.evaluate` and `tf.keras.Model.predict` methods can use NumPy
data and a `tf.data.Dataset`.

Here's how to *evaluate* the inference-mode loss and metrics for the data provided:

In [10]:
# With Numpy arrays
data = np.random.random((1000, 32))
labels = np.random.random((1000, 10))

model.evaluate(data, labels, batch_size=32)

# With a Dataset
dataset = tf.data.Dataset.from_tensor_slices((data, labels))
dataset = dataset.batch(32)

model.evaluate(dataset)

32/32 [==============================] - 0s 4ms/step - loss: 433274.7881 - acc: 0.1240


[433274.7880859375, 0.124]

And here's how to *predict* the output of the last layer in inference for the data provided,
as a NumPy array:

In [11]:
result = model.predict(data, batch_size=32)
print(result.shape)

(1000, 10)


For a complete guide on training and evaluation, including how to write custom training loops from scratch, see the [guide to training and evaluation](./train_and_evaluate.ipynb).

## Build complex models

### The Functional API

 The `tf.keras.Sequential` model is a simple stack of layers that cannot
represent arbitrary models. Use the
[Keras functional API](./functional.ipynb)
to build complex model topologies such as:

* Multi-input models,
* Multi-output models,
* Models with shared layers (the same layer called several times),
* Models with non-sequential data flows (e.g. residual connections).

Building a model with the functional API works like this:

1. A layer instance is callable and returns a tensor.
2. Input tensors and output tensors are used to define a `tf.keras.Model`
   instance.
3. This model is trained just like the `Sequential` model.

The following example uses the functional API to build a simple, fully-connected
network:

In [13]:
inputs = tf.keras.Input(shape=(32,))  # Returns an input placeholder

# A layer instance is callable on a tensor, and returns a tensor.
x = layers.Dense(64, activation='relu')(inputs)
x = layers.Dense(64, activation='relu')(x)
predictions = layers.Dense(10)(x)

Instantiate the model given inputs and outputs.

In [14]:
model = tf.keras.Model(inputs=inputs, outputs=predictions)

# The compile step specifies the training configuration.
model.compile(optimizer=tf.keras.optimizers.RMSprop(0.001),
              loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

# Trains for 5 epochs
model.fit(data, labels, batch_size=32, epochs=5)

Train on 1000 samples
Epoch 1/5
1000/1000 [==============================] - 0s 383us/sample - loss: 12.9059 - accuracy: 0.0890
Epoch 2/5
1000/1000 [==============================] - 0s 72us/sample - loss: 21.3660 - accuracy: 0.0880
Epoch 3/5
1000/1000 [==============================] - 0s 74us/sample - loss: 34.6242 - accuracy: 0.1020
Epoch 4/5
1000/1000 [==============================] - 0s 77us/sample - loss: 46.5124 - accuracy: 0.1100
Epoch 5/5
1000/1000 [==============================] - 0s 73us/sample - loss: 60.8220 - accuracy: 0.0840
